# Day 2 EXERCISE Solution:

Upgraded day 1 project that scrapes and summarizes any webpage using an Open Source model running locally via Ollama instead of OpenAI

## Note:-
If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative: 
1. Run `ollama pull llama3.2:1b` from a Terminal or Powershell
2. **Ctrl + /** to comment this code line below: `MODEL = "llama3.2"`
3. same **Ctrl + /** to uncomment: `MODEL = "llama3.2:1b"`

In [ ]:
# imports:-

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
# Constants:-

# MODEL = "llama3.2"
MODEL = "llama3.2:1b"
# MODEL = "deepseek-r1:1.5b"

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


system_prompt = "You are an assistant that analyzes the contents of a website \
                and provides a short summary, ignoring text that might be navigation related. \
                Respond in markdown."


def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


def summary(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return display(Markdown(response['message']['content']))


summary("https://edwarddonner.com")
# summary("https://cnn.com")
# summary("https://anthropic.com")